In [ ]:
import os
import json
import requests
import pandas as pd

# =========================
# 0) 공통 설정
# =========================
CLIENT_ID = "CLIENT_ID"
CLIENT_SECRET = "CLIENT_SECRET"

headers = {
    "CLIENT_ID": CLIENT_ID,
    "CLIENT_SECRET": CLIENT_SECRET,
}

BASE_URL = "https://openapi.naver.com/v1/search/book.json"


# =========================
# 1) 공통 요청 함수 (중복 제거)
# =========================
def _naver_get(url: str, params: dict, headers: dict) -> dict:
    res = requests.get(url, params=params, headers=headers, timeout=15)

    # ✅ 실패하면 네이버가 준 응답을 그대로 보여주기
    if res.status_code != 200:
        print("STATUS:", res.status_code)
        print("URL:", res.url)
        print("HEADERS CHECK:",
              "id_len=", len(headers.get("X-Naver-Client-Id", "")),
              "secret_len=", len(headers.get("X-Naver-Client-Secret", "")))
        print("RAW TEXT:", res.text)  # 여기서 errorCode / errorMessage가 나옵니다
        res.raise_for_status()

    return res.json()



# =========================
# 2) 도서 검색 함수 (query 인자 동적 처리)
# =========================
def search_books(query: str, display: int = 100, sort: str = "sim") -> list[dict]:
    payload = {
        "query": query,
        "display": display,
        "sort": sort,
    }
    data = _naver_get(BASE_URL, payload, headers)
    return data.get("items", [])


# =========================
# 3) JSON 저장/로드
# =========================
def save_json(items: list[dict], path: str) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(items, f, ensure_ascii=False, indent=2)


def load_json(path: str) -> list[dict]:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


# =========================
# 4) DataFrame 변환/정리
# =========================
def to_dataframe(items: list[dict]) -> pd.DataFrame:
    df = pd.DataFrame(items)

    # discount(가격) 문자열 -> 숫자 변환
    if "discount" in df.columns:
        df["discount"] = pd.to_numeric(df["discount"], errors="coerce")

    return df


# =========================
# 5) 과제 실행
# =========================
if __name__ == "__main__":
    query = "파이썬"  # 여기만 바꾸면 동적으로 검색됨

    # 3-1. 네이버 책 검색 API 호출
    items = search_books(query)

    # (1) 검색 결과를 json 파일로 저장
    save_path = "data/books.json"
    save_json(items, save_path)
    print(f"[1) 저장 완료] {save_path} (총 {len(items)}권)")

    # (2) books.json 파일을 Pandas DataFrame로 저장하기 (파일을 읽어서 DF 생성)
    items_from_file = load_json(save_path)
    df = to_dataframe(items_from_file)

    # (3) 검색어로 찾은 책 목록 출력하기
    print("\n[3) 검색 결과 목록 상위 10개]")
    print(df[["title", "author", "discount", "publisher", "pubdate"]].head(10))

    # (4) 가격이 2만원 이상인 책만 출력하기
    # title, author, discount, publisher, pubdate 컬럼만 출력
    # 가격 내림차순, index 초기화
    cols_4 = ["title", "author", "discount", "publisher", "pubdate"]
    df_20k = (
        df.loc[df["discount"].fillna(0) >= 20000, cols_4]
          .sort_values("discount", ascending=False)
          .reset_index(drop=True)
    )
    print("\n[4) 가격 2만원 이상 도서 (가격 내림차순)]")
    print(df_20k)

    # (5) 출판사가 “인피니티북스”인 책만 출력하기
    # image, description 컬럼 제외한 모든 컬럼 출력
    # index 초기화
    drop_cols = [c for c in ["image", "description"] if c in df.columns]
    df_inf = (
        df.loc[df["publisher"] == "인피니티북스"]
          .drop(columns=drop_cols, errors="ignore")
          .reset_index(drop=True)
    )
    print("\n[5) 출판사=인피니티북스 도서 목록 (image/description 제외)]")
    print(df_inf)


HTTPError: 401 Client Error: Unauthorized for url: https://openapi.naver.com/v1/search/book.json?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC&display=100&sort=sim